In [1]:
using RigidBodyDynamics, RigidBodyDynamics.Contact, RigidBodyDynamics.OdeIntegrators
using MomentumBasedControl, MomentumBasedControl.PDControl
using StaticArrays
using RigidBodyTreeInspector, DrakeVisualizer
using ValkyrieRobot
using BipedControlUtil
using ForwardDiff

In [2]:
val = Valkyrie()
mechanism = val.mechanism;

In [3]:
# add environment
rootframe = root_frame(mechanism)
floor = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
add_environment_primitive!(mechanism, floor);

In [4]:
# Create visualizer
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1))
vis = Visualizer(parse_urdf(ValkyrieRobot.urdfpath(), mechanism; package_path = [ValkyrieRobot.packagepath()]), :valkyrie);

In [5]:
remove_fixed_tree_joints!(mechanism)
const state = MechanismState(Float64, mechanism)
const controllerstate = MomentumBasedControllerState(state);

In [6]:
# Initial setup
zero!(state)
kneebend = 1.1
hipbendextra = 0.1
for side in instances(Side)
    set_configuration!(state, val.knees[side], [kneebend])
    set_configuration!(state, val.hippitches[side], [-kneebend / 2 + hipbendextra])
    set_configuration!(state, val.anklepitches[side], [-kneebend / 2 - hipbendextra])
end
set_configuration!(state, val.floatingjoint, [1; 0; 0; 0; 0; 0; 1.05]) # TODO
settransform!(vis, state)
MomentumBasedControl.reset!(controllerstate);

In [7]:
const τ = Vector{Float64}(num_velocities(mechanism))
const result = DynamicsResult(Float64, mechanism)
function damped_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
    τ[:] = velocity(state)
    scale!(τ, -1.0)
    dynamics!(result, state, τ)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ)
end

damped_dynamics! (generic function with 1 method)

In [8]:
const controller = MomentumBasedController{Float64}(mechanism, 5e-3);
contacts = add_mechanism_contacts!(controller)
jointacceltasks = add_mechanism_joint_accel_tasks!(controller);

In [9]:
comgains = PDGains(10., 3.)

MomentumBasedControl.PDControl.PDGains{Float64}(10.0,3.0)

In [10]:
# Foot accelerations w.r.t. world
footacceltasks = Dict{Side, SpatialAccelerationTask{Float64}}()
legjoints = []
for side in instances(Side)
    foot = val.feet[side]
    world_to_foot = path(mechanism, root_body(mechanism), foot)
    pelvis_to_foot = path(mechanism, val.pelvis, foot)
    append!(legjoints, joint for (joint, direction) in pelvis_to_foot)
    frame = default_frame(foot)
    task = footacceltasks[side] = SpatialAccelerationTask(world_to_foot, frame, eye(3, 3), eye(3, 3))
    add!(controller, task)
end

In [11]:
# Pelvis acceleration w.r.t. world
pelvisacceltask = SpatialAccelerationTask(path(mechanism, root_body(mechanism), val.pelvis), default_frame(val.pelvis), eye(3, 3), zeros(0, 3))
add!(controller, pelvisacceltask)
pelvisgains = PDGains(10., 2.)
;

In [12]:
#Position control tasks
revolutejoints = filter(j -> isa(j.jointType, Revolute), joints(mechanism))
positioncontroltasks = Dict(joint => jointacceltasks[joint] for joint in setdiff(revolutejoints, legjoints))
positioncontrolgains = Dict(joint => PDGains(100.0, 20.) for joint in keys(positioncontroltasks))
;

In [13]:
# Position control references
references = Dict{Joint{Float64}, Any}(joint => zero for joint in keys(positioncontroltasks))
references[findjoint(mechanism, "rightShoulderRoll")] = t -> π/2 - 0.1
references[findjoint(mechanism, "rightShoulderYaw")] = t -> 1.
references[findjoint(mechanism, "leftElbowPitch")] = t -> -0.4
references[findjoint(mechanism, "leftWristPitch")] = t -> 0.5
references[findjoint(mechanism, "leftShoulderRoll")] = t -> 0.9 + 0.3 * sin(t)

(::#19) (generic function with 1 method)

In [14]:
for (joint, ref) in references
    configuration(state, joint)[:] = ref(0.)
    velocity(state, joint)[:] = ForwardDiff.derivative(ref, 0.)
end
setdirty!(state)
settransform!(vis, state)

Set{Array{Symbol,1}}()

In [15]:
feet = val.feet
com0 = center_of_mass(state)
com0 += FreeVector3D(com0.frame, 0.0, 0.0, -0.05)
μ = 0.8
normal = floor.outward_normal
i = 0
function controlled_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
    # TODO: extract out: highlevelcontrol
    # Foot accelerations
    T = eltype(controller)
    MomentumBasedControl.reset!(controller)
    regularize_joint_accels!(controller, 0.05)
    for (side, task) in footacceltasks
        zero!(task, Inf)
        
        body = RigidBodyDynamics.Graphs.target(task.path)
        for contactsettings in contacts[body]
            set!(contactsettings, 1e-1, μ, transform(state, normal, default_frame(body)))
        end
    end
    
    # Pelvis orientation control
    Hpelvis = transform_to_root(state, val.pelvis)
    Tpelvis = transform(twist_wrt_world(state, val.pelvis), inv(Hpelvis))
    ωdpelvis = pd(pelvisgains, rotation(Hpelvis), Tpelvis.angular)
    Ṫpelvis = SpatialAcceleration(Tpelvis.body, Tpelvis.base, Tpelvis.frame, ωdpelvis, zero(ωdpelvis))
    set!(pelvisacceltask, Ṫpelvis, Inf)
    
    # Joint position control
    for (joint, task) in positioncontroltasks
        gains = positioncontrolgains[joint]
        ref = references[joint]
        qjoint, vjoint, vdjoint = val_deriv_deriv2(ref, t)
        accel = vdjoint = pd(gains, configuration(state, joint), qjoint, velocity(state, joint), vjoint)
        set!(task, accel, Inf)
    end
    
    # Centroidal momentum control
    pd_center_of_mass!(controller, comgains, state, com0, FreeVector3D(T, com0.frame), 1.0)

    τ = control(controller, t, controllerstate)
    dynamics!(result, state, τ)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ)
end

controlled_dynamics! (generic function with 1 method)

In [16]:
# Simulation
const dyn! = controlled_dynamics!
# const dyn! = damped_dynamics!
# sink = DrakeVisualizerSink(vis)
const sink = ExpandingStorage{Float64}(10000)
const integrator = MuntheKaasIntegrator(dyn!, runge_kutta_4(Float64), sink);

In [17]:
integrate(integrator, state, 10., 1e-4, maxRealtimeRate = Inf)

In [20]:
animate(vis, mechanism, sink.ts, sink.qs)